In [764]:
import matplotlib.pyplot as plt
plt.rcParams['image.interpolation'] = 'none'
import numpy as np
import cv2
import math
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from scipy.ndimage import gaussian_filter1d
import itertools
import random
from sklearn.linear_model import RANSACRegressor, LinearRegression
import torch
import pandas as pd
import warnings
import seaborn as sns
warnings.simplefilter(action='ignore', category=FutureWarning)
import random
random.seed(42)

In [765]:
def process_image(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = model(img)
    detections = results.pandas().xyxy[0]
    results.render()

    traffic_lights = []
    zebra_crops = []

    for _, row in detections.iterrows():
        class_name = row['name']
        xmin, ymin, xmax, ymax = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        coords = (xmin, ymin, xmax, ymax)
        confidence = round(float(row['confidence']), 2)

        if class_name == 'Zebra_Cross':
            zebra_crops.append((coords, confidence))
        elif class_name == 'G_Signal':
            traffic_lights.append({'coords': coords, 'confidence': confidence, 'color': 'green'})
        elif class_name == 'R_Signal':
            traffic_lights.append({'coords': coords, 'confidence': confidence, 'color': 'red'})

    return results, traffic_lights, zebra_crops

In [766]:
def get_results(row, ptl_list, zebra_crops, zebra, red, green):
    if row['zebra'] == 1:  # Ground truth: Zebra crossing present
        if zebra_crops:  # Prediction: Zebra crossing detected
            zebra[0] += 1  # TP
        else:  # Prediction: Zebra crossing not detected
            zebra[2] += 1  # FN
    else:  # Ground truth: No zebra crossing
        if zebra_crops:  # Prediction: Zebra crossing detected
            zebra[1] += 1  # FP
        else:  # Prediction: Zebra crossing not detected
            zebra[3] += 1  # TN

    if ptl_list:  # Traffic light(s) detected
        if row['mode'] in [0, 3]:  # Ground truth: Red light should be present
            if ptl_list[0]['color'] == 'red':
                red[0] += 1  # TP
            else:
                red[2] += 1  # FN
        else:  # Ground truth: Red light should NOT be present
            if ptl_list[0]['color'] == 'red':
                red[1] += 1  # FP
            else:
                red[3] += 1  # TN

        if row['mode'] in [1, 2]:  # Ground truth: Green light should be present
            if ptl_list[0]['color'] == 'green':
                green[0] += 1  # TP
            else:
                green[2] += 1  # FN
        else:  # Ground truth: Green light should NOT be present
            if ptl_list[0]['color'] == 'green':
                green[1] += 1  # FP
            else:
                green[3] += 1  # TN
    else:  # No traffic lights detected
        if row['mode'] in [0, 3]:  # Ground truth: Red light should be present
            red[1] += 1  # FP (Model didn't detect it)
        else:  # Ground truth: Red light should NOT be present
            red[3] += 1  # TN (Model correctly didn't detect it)

        if row['mode'] in [1, 2]:  # Ground truth: Green light should be present
            green[1] += 1  # FP (Model didn't detect it)
        else:  # Ground truth: Green light should NOT be present
            green[3] += 1  # TN (Model correctly didn't detect it)

    return zebra, red, green

In [767]:
def calculate_metrics(type):
    total = sum(type)
    tp = type[0]
    fp = type[1]
    fn = type[2]
    tn = type[3]

    accuracy = (tp + tn) / total if total > 0 else 0

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'accuracy': round(accuracy, 2),
        'precision': round(precision, 2),
        'recall': round(recall, 2),
        'f1_score': round(f1_score, 2)
    }

In [768]:

def plot_confusion_matrices_with_metrics(zebra, red, green, zebra_metrics, red_metrics, green_metrics, filename="Matriu de confusió.png"):
    clases = ['Pas de zebra', 'Semàfor vermell', 'Semàfor verd']
    datos = [zebra, red, green]
    all_metrics = [zebra_metrics, red_metrics, green_metrics]

    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    for ax, clase, (tp, fp, fn, tn), metrics in zip(axs, clases, datos, all_metrics):
        matrix = np.array([[tp, fn],
                           [fp, tn]])

        sns.heatmap(matrix, annot=True, fmt="d", cmap="Reds", cbar=False, ax=ax, xticklabels=['Positiu', 'Negatiu'], yticklabels=['Positiu', 'Negatiu'], annot_kws={"size": 15})

        ax.set_title(f'{clase}')
        ax.set_xlabel('Predicció')
        ax.set_ylabel('Real')

        metrics_text = f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1: {metrics['f1_score']}"
        ax.text(0.05, -0.3, metrics_text, size=18, ha='left', va='top', transform=ax.transAxes) 

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) 
    plt.savefig(filename)
    plt.close(fig)


In [769]:
def display_images(imgs, titles, width=5):
    n = len(imgs)
    rows = math.ceil(n / 3)
    cols = n // rows
    h, w = imgs[0].shape[:2]
    fig, axs = plt.subplots(rows, cols, figsize=(width * cols, width * h / w * rows))
    axs = axs.flatten()
    for i in range(rows * cols):
        if i < n:
            img = imgs[i]
            axs[i].imshow(img, cmap='gray' if img.ndim == 2 else None)
            axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


def to_float(img):
    img = img.astype(np.float64)
    return (img - np.min(img)) / (np.max(img) - np.min(img))

def to_uint8(img):
    norm = (img - np.min(img)) / (np.max(img) - np.min(img))
    return np.round(norm * 255).astype(np.uint8)

In [770]:
def noise_filter(src, option='median', med_k=5, gauss_sigma=1.5, bilat_d=6, bilat_sigmaColor=75, bilat_sigmaSpace=75, morph_kernel=5, **kwargs):
    if option == 'median':
        return cv2.medianBlur(src, med_k)
    elif option == 'gaussian':
        ksize = round(6 * gauss_sigma + 1)
        ksize += 1 - (ksize % 2)
        return cv2.GaussianBlur(src, (ksize, ksize), gauss_sigma)
    elif option == 'bilateral':
        return cv2.bilateralFilter(src, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)
    elif option == 'morph_max':
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (morph_kernel, morph_kernel))
        return cv2.morphologyEx(src, cv2.MORPH_DILATE, kernel)
    else:
        raise ValueError(f"Invalid filter option: {option}")

In [771]:
def find_threshold(src, option='mixture', **kwargs):
    if option == 'otsu':
        thr, _ = cv2.threshold(src, 0, 256, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thr
    elif option == 'mixture':
        hist, bins = np.histogram(src, bins=256, range=(0, 256))
        sample = cv2.resize(src, (14, 14), interpolation=cv2.INTER_CUBIC).reshape(-1, 1)
        gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
        gmm.fit(sample)
        means = gmm.means_.flatten()
        m1, m2 = np.sort(means)
        hist_smooth = gaussian_filter1d(hist, sigma=2)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        mask = (bin_centers > m1) & (bin_centers < m2)
        thr = bin_centers[mask][np.argmin(hist_smooth[mask])]
        return thr
    else:
        raise ValueError(f"Invalid thresholding option: {option}")

In [772]:
def get_edges(src, option="laplacian", canny_thr1=50, canny_thr2=150, **kwargs):
    if option == "canny":
        return cv2.Canny(src, canny_thr1, canny_thr2, apertureSize=3)
    elif option == "laplacian":
        src_blur = cv2.GaussianBlur(src, (5,5), 0)
        lap = cv2.Laplacian(src_blur, ddepth=cv2.CV_64F)
        z1 = np.roll(lap, 1, axis=0) * np.roll(lap, -1, axis=0)
        z2 = np.roll(lap, 1, axis=1) * np.roll(lap, -1, axis=1)
        return np.logical_or(z1 < 0, z2 < 0).astype(np.uint8) * 255
    else:
        raise ValueError(f"Invalid edges option: {option}")

In [773]:
def get_intersection(l1, l2):
    rho1, theta1 = l1
    rho2, theta2 = l2
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    if np.abs(np.linalg.det(A)) < 1e-6:
        return None
    intersection = np.linalg.solve(A, b)
    return intersection.flatten()

def point_line_distance(x0, y0, rho, theta):
    return abs(x0 * np.cos(theta) + y0 * np.sin(theta) - rho)

In [774]:
def get_filtered_lines(edges, high_thr=240, low_thr=150, centroids_sep=100, line_dist_thr=30, **kwargs):
    few_raw = cv2.HoughLines(edges, 1, np.pi/180, threshold=high_thr)
    raw = cv2.HoughLines(edges, 1, np.pi/180, threshold=low_thr)
    if few_raw is None or raw is None:
        return []
    few_lines = few_raw[:, 0, :]
    lines = raw[:, 0, :]

    intersections = []
    for l1, l2 in itertools.combinations(few_lines, 2):
        pt = get_intersection(l1, l2)
        if pt is not None and np.all(np.isfinite(pt)):
            intersections.append(pt)
    if not intersections:
        return few_lines
    intersections = np.array(intersections)

    clustering = DBSCAN(eps=40, min_samples=5).fit(intersections)
    labels = clustering.labels_
    if np.sum(labels != -1) == 0:
        return few_lines

    unique_labels = np.unique(labels[labels != -1])
    cluster_sizes = np.array([np.sum(labels == k) for k in unique_labels])
    centroids = np.stack([intersections[labels == k].mean(axis=0) for k in unique_labels])
    centroids = centroids[np.argsort(cluster_sizes)[::-1]]

    points = []
    for centroid in centroids:
        if all(centroid[0]-p[0] > centroids_sep and centroid[1]-p[1] > centroids_sep for p in points):
            points.append(centroid)
    points = np.array(points)

    filtered_lines = []
    for rho, theta in lines:
        for x, y in points:
            if point_line_distance(x, y, rho, theta) < line_dist_thr:
                filtered_lines.append([rho, theta])
                break
    filtered_lines = np.array(filtered_lines)

    return filtered_lines

def remove_duplicate_lines(lines, shape):
    if len(lines) == 0:
        return np.array([])

    lines = np.array(lines)
    h, w = shape[:2]
    d = np.hypot(h, w)
    theta_mean = np.mean(lines[:, 1])
    keep = np.ones(len(lines), dtype=bool)

    def get_line_points(rho, theta):
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = np.array([x0 + d * -b, y0 + d * a])
        pt2 = np.array([x0 - d * -b, y0 - d * a])
        return pt1, pt2

    def intersection(p1, p2, q1, q2):
        A = np.array([[p2[0] - p1[0], q1[0] - q2[0]],
                      [p2[1] - p1[1], q1[1] - q2[1]]])
        b = np.array([q1[0] - p1[0], q1[1] - p1[1]])
        if np.linalg.matrix_rank(A) < 2:
            return None
        t = np.linalg.solve(A, b)
        return p1 + t[0] * (p2 - p1)

    for i in range(len(lines)):
        if not keep[i]:
            continue
        rho1, theta1 = lines[i]
        p1, p2 = get_line_points(rho1, theta1)
        for j in range(i + 1, len(lines)):
            if not keep[j]:
                continue
            rho2, theta2 = lines[j]
            q1, q2 = get_line_points(rho2, theta2)
            inter = intersection(p1, p2, q1, q2)
            if inter is not None:
                x, y = inter
                if 0 <= x < w and 0 <= y < h:
                    dist_i = abs(theta1 - theta_mean)
                    dist_j = abs(theta2 - theta_mean)
                    if dist_i > dist_j:
                        keep[i] = False
                        break
                    else:
                        keep[j] = False

    return lines[keep]

In [775]:
def segment_from_rho_theta(rho, theta, edge_img, epsilon=1.0):

    ys, xs = np.nonzero(edge_img) 
    pts = np.stack([xs, ys], axis=1).astype(np.float32)

    n = np.array([np.cos(theta), np.sin(theta)]) 
    v = np.array([-np.sin(theta),  np.cos(theta)])
    d = np.abs(pts @ n - rho)
    on_line = pts[d < epsilon]

    if len(on_line) < 2:
        return None
    
    t = on_line @ v
    p_min = on_line[np.argmin(t)]
    p_max = on_line[np.argmax(t)]
    
    return tuple(p_min.astype(int)), tuple(p_max.astype(int))


In [776]:
def count_near_points(p1, p2, pts, dist):
    pts = np.asarray(pts, dtype=float)
    p1  = np.asarray(p1,  dtype=float)
    p2  = np.asarray(p2,  dtype=float)

    v = p2 - p1
    norm_v = np.linalg.norm(v)

    if norm_v == 0:
        raise ValueError("p1 y p2 no poden coincidir.")
    dif   = p1 - pts
    cross = v[0] * dif[:, 1] - v[1] * dif[:, 0]

    dists = np.abs(cross) / norm_v
    return np.sum(dists < dist)


In [777]:
def get_line_ransac(points, ransac_dist=20, ransac_n_iter=100, **kwargs):
    n_lines = len(points)
    best_p=(0,0)
    best_k=0
    if points is not None:
            for i in range(ransac_n_iter):
                p1 = points[random.randint(0, n_lines-1)]
                p2 = points[random.randint(0, n_lines-1)]
                while np.all(p1 == p2):
                    p2 = points[random.randint(0, n_lines-1)]
                k = count_near_points(p1,p2, points, ransac_dist)
                if k > best_k:
                    best_k = k
                    best_p = (p1, p2)
    return best_p

In [778]:
def intersect_polar_with_segment(rho, theta, segment, eps=1e-6):
    (x1, y1), (x2, y2) = segment
    dx, dy = x2 - x1, y2 - y1
    denom = dx*np.cos(theta) + dy*np.sin(theta)
    if abs(denom) < eps:
        return None
    t = (rho - (x1*np.cos(theta) + y1*np.sin(theta))) / denom
    x = x1 + t*dx
    y = y1 + t*dy
    return (int(round(x)), int(round(y)))

In [779]:
def get_limits(representative_lines, edges, ransac_dist=20, ransac_n_iter=100, **kwargs):
    lista_extremos = []
    for (rho, theta) in representative_lines:
        extremos = segment_from_rho_theta(rho, theta, edges, epsilon=1.5)
        if extremos is not None:
            lista_extremos.append(extremos)
    lista_extremos = np.array(lista_extremos)

    # Retornem els límits de la imatge en cas de no trobar múltiples extrems
    if len(lista_extremos) <= 1:
        h, w = edges.shape
        return ((0,0), (0,h)), ((w,0), (w,h))

    izq = lista_extremos[:, 0]
    der = lista_extremos[:, 1]
    p_izq=get_line_ransac(izq, ransac_dist, ransac_n_iter)
    p_der=get_line_ransac(der, ransac_dist, ransac_n_iter)

    return p_izq, p_der

In [780]:
def get_mid_points(lines, p_izq, p_der, shape):

    lista_midpoints = []
    h, w = shape

    for rho, theta in lines:
        sin_t, cos_t = np.sin(theta), np.cos(theta)
        if abs(sin_t) < 1e-6:
            continue
        pi = intersect_polar_with_segment(rho, theta, p_izq)
        pd = intersect_polar_with_segment(rho, theta, p_der)

        # Calcul i filtrat de interseccions
        border = []
        y0 = (rho - 0*cos_t)/sin_t
        yw = (rho - w*cos_t)/sin_t
        if 0 <= y0 <= h: border.append((0, int(round(y0))))
        if 0 <= yw <= h: border.append((w, int(round(yw))))
        x0 = (rho - 0*sin_t)/cos_t if abs(cos_t)>1e-6 else None
        xh = (rho - h*sin_t)/cos_t if abs(cos_t)>1e-6 else None
        if x0 is not None and 0 <= x0 <= w: border.append((int(round(x0)), 0))
        if xh is not None and 0 <= xh <= w: border.append((int(round(xh)), h))
        if len(border) < 2:
            continue
        
        border = sorted(border, key=lambda p: p[0])
        start_border, end_border = border[0], border[-1]

        # Seleccio de punts: interseccions si existeixen, si no, els extrems de la imatge
        start = pi if pi is not None else start_border
        end   = pd if pd is not None else end_border

        # Calcul de punt mitjà
        mx = int(round((start[0] + end[0]) / 2))
        my = int(round((start[1] + end[1]) / 2))
        lista_midpoints.append((mx, my))

    return lista_midpoints


In [781]:
def get_angle(midpoints, shape):
    h, w = shape
    pts = np.array(midpoints)
    xs, ys = pts[:,0], pts[:,1]
    X = xs.reshape(-1,1)
    y = ys

    ransac = RANSACRegressor(LinearRegression(), 
                            residual_threshold = w * 0.005,
                            max_trials=100, random_state=42)
    ransac.fit(X, y)

    m = ransac.estimator_.coef_[0]

    angle_rad = np.pi/2 - np.arctan(m)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg

In [782]:
def get_init_point(midpoints):
    return max(midpoints, key=lambda p: p[1])

## Evaluació del model

In [783]:
def predict_img(img_path, model, **kwargs):
    path = '../data'+img_path
    results, traffic_lights, zebra_crops = process_image(path, model)
    if not zebra_crops:
            return pd.Series([img_path, 0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    
    try:
        mode = 0
        if blocked := len(traffic_lights) != 0:
            mode = traffic_lights[0]["color"] == "green"
        
        (xmin, ymin, xmax, ymax) = zebra_crops[0][0]

        img = plt.imread(path)
        img = img[ymin:ymax, xmin:xmax]
        gray = to_uint8(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
        cv2.imwrite("./actual.jpg",img)
        # print(path)

        src = noise_filter(gray, **kwargs)
        thr = find_threshold(src, **kwargs)
        bw = 255*(src > thr).astype(np.uint8)
        edges = get_edges(bw, **kwargs)
        filtered_lines = get_filtered_lines(edges, **kwargs)
        filtered_lines = remove_duplicate_lines(filtered_lines, gray.shape)
        if len(filtered_lines) == 0:
             raise Exception("No s'han trobat línies")

        p_izq, p_der = get_limits(filtered_lines, edges, **kwargs)
        midpoints = get_mid_points(filtered_lines, p_izq, p_der, gray.shape)
        angle_rad, angle_deg = get_angle(midpoints, gray.shape)
        x, y = get_init_point(midpoints)

        return pd.Series([img_path, 1, mode, blocked, x, y, angle_rad, angle_deg])
    
    except Exception as e:
        # print("Error:", e)
        print("Error: ", e, " Imatge: ", path, " Thr: ", thr)
        # cv2.imwrite("./edges.jpg", src)
        # return pd.Series([img_path, 0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
        # raise e

In [784]:
model = torch.hub.load('yolov5', 'custom', path='best.pt', source='local')
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv('../data/dataset.csv')

df_pred = df.progress_apply(lambda row: predict_img(row["file"], model), axis=1)


YOLOv5  2025-5-16 Python-3.10.11 torch-2.6.0+cpu CPU

Fusing layers... 
Model summary: 280 layers, 12315904 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 
  0%|          | 22/6955 [00:06<30:25,  3.80it/s]

Error:  No s'han trobat línies  Imatge:  ../data/pedestrian-traffic-lights/sam_IMG_1490.jpg  Thr:  36.5


  1%|          | 36/6955 [00:10<29:36,  3.89it/s]

Error:  No s'han trobat línies  Imatge:  ../data/pedestrian-traffic-lights/heon_IMG_0987.JPG  Thr:  64.5


  1%|          | 66/6955 [00:19<29:37,  3.88it/s]

...

Error:  `min_samples` may not be larger than number of samples: n_samples = 1.  Imatge:  ../data/pedestrian-traffic-lights/heon_IMG_0828.JPG  Thr:  66.5


 16%|█▋        | 1146/6955 [06:33<26:13,  3.69it/s]

Error:  No s'han trobat línies  Imatge:  ../data/pedestrian-traffic-lights/heon_IMG_0722.JPG  Thr:  58.5


 16%|█▋        | 1146/6955 [06:34<33:17,  2.91it/s]


UnboundLocalError: local variable 'thr' referenced before assignment

In [ ]:
# model = torch.hub.load('yolov5', 'custom', path='best.pt', source='local')
# df = pd.read_csv('../data/dataset.csv')

# path = ""
# predict_img(path, model)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error
import ace_tools as tools

assert df.shape[0] == df_pred.shape[0]
classification_metrics = {}
for col in ['mode', 'blocked']:
    y_true = df[col].dropna()
    y_pred = df_pred.loc[y_true.index, col].dropna()
    valid_idx = y_true.index.intersection(y_pred.index)
    classification_metrics[col] = {
        'accuracy': accuracy_score(df.loc[valid_idx, col], df_pred.loc[valid_idx, col]),
        'precision': precision_score(df.loc[valid_idx, col], df_pred.loc[valid_idx, col], zero_division=0),
        'recall': recall_score(df.loc[valid_idx, col], df_pred.loc[valid_idx, col], zero_division=0),
        'f1': f1_score(df.loc[valid_idx, col], df_pred.loc[valid_idx, col], zero_division=0),
        'confusion_matrix': confusion_matrix(df.loc[valid_idx, col], df_pred.loc[valid_idx, col])
    }

# MAE per a valors continus
mae_metrics = {}
for col in ['x', 'y', 'theta_rad', 'theta_deg']:
    y_true = df[col]
    y_pred = df_pred[col]
    valid = ~(y_true.isna() | y_pred.isna())
    mae_metrics[col] = mean_absolute_error(y_true[valid], y_pred[valid]) if valid.any() else np.nan

# Convertim a DataFrame per visualitzar
summary_classification = pd.DataFrame({
    metric: {
        col: val[metric] for col, val in classification_metrics.items()
    } for metric in ['accuracy', 'precision', 'recall', 'f1']
})

summary_mae = pd.Series(mae_metrics, name='MAE').to_frame()

tools.display_dataframe_to_user(name="Classification Metrics", dataframe=summary_classification)
tools.display_dataframe_to_user(name="MAE Metrics", dataframe=summary_mae)

# Dibuixem les matrius de confusió
for col in ['mode', 'blocked']:
    cm = classification_metrics[col]['confusion_matrix']
    plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {col}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()